In [34]:
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
from sklearn.model_selection import train_test_split


In [35]:
import pandas as pd
mobile = pd.read_csv('Data/mobile_cleaned.csv')
# mobile.drop(columns='time',inplace=True)
mobile.drop_duplicates(inplace=True)
mobile = mobile.reset_index(drop=True)
mobile.reset_index(drop=True, inplace=True)

print(mobile.mobile_strength.min())
print(mobile.mobile_strength.max())

mobile.mobile_strength = mobile.mobile_strength.div(10)
mobile

13
62


,time,lat,long,mobile_strength
0,1712870250,40.112640,-88.218734,3.6
1,1712870260,40.112646,-88.218679,3.6
2,1712870292,40.112550,-88.218558,2.9
3,1712870292,40.112550,-88.218558,3.9
4,1712870300,40.112630,-88.218658,2.9
...,...,...,...,...
2648,1714183561,40.108236,-88.235220,4.8
2649,1714183561,40.108236,-88.235220,5.0
2650,1714183561,40.108236,-88.235220,4.4
2651,1714183561,40.108236,-88.235220,4.7


In [36]:
from sklearn.model_selection import train_test_split
X = mobile[['time', 'lat', 'long']]  # Features: time, latitude, longitude
y = mobile['mobile_strength'] # y val
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.001, random_state=42) # keeping test really low since we are not evaluating


In [37]:
model = RandomForestRegressor() # decision tree based regression suitable for our use case (Given gps coordinates, predict signal strength) 
model.fit(X_train, y_train)


y_pred = model.predict(X_test) # sample testing


df = pd.DataFrame({'lat':X_test.lat,'long':X_test.long,'expected':y_test,'predicted':y_pred}).reset_index(drop=True) 
df

,lat,long,expected,predicted
0,40.103832,-88.234731,2.4,2.4018
1,40.112556,-88.218597,3.0,3.0000
2,40.110484,-88.219126,3.0,3.0000


In [44]:
from sklearn.neighbors import KDTree

test = df.iloc[0]
threshold  = 1 # used to select nearest coordinate with signal strength diff higher than thresh


higher_signal_points = mobile[mobile.mobile_strength>=test.predicted+threshold].reset_index(drop=True)[['lat','long']]

kd_tree = KDTree(higher_signal_points) # an efficient structure for representing spatial data

test_coords = [[test.lat, test.long]]

distance, nearest_index = kd_tree.query(test_coords, k=1) # find the nearest coordinate based on the filter

print(distance[0], nearest_index[0])

nearest_point = higher_signal_points.iloc[nearest_index[0][0]]

print(test)
print(mobile[(mobile.lat == nearest_point.lat) & (mobile.long == nearest_point.long)].iloc[0])

[0.00169947] [357]
lat          40.103832
long        -88.234731
expected      2.400000
predicted     2.401800
Name: 0, dtype: float64
time               1.714183e+09
lat                4.010550e+01
long              -8.823508e+01
mobile_strength    4.100000e+00
Name: 2547, dtype: float64
